In [1]:
import sys
sys.path.append("../../src/")
import uncertainpy.gradual as grad
import random
random.seed(0)

In [2]:
from itertools import combinations

# generate random combination with number n
def combine(temp_list, n):
    return list(combinations(temp_list, n))

In [3]:
# generate random sublist from current list
def generate_new_list(original_list):
    new_list = []
    for element in original_list:
        if random.choice([True, False]):
            new_list.append(element)
    return new_list

In [4]:
# set semantics and compute strength of arguments
bag = grad.BAG("../../bags/fig6.bag")

# DF-QuAD semantics
agg_f = grad.semantics.modular.ProductAggregation()
inf_f = grad.semantics.modular.LinearInfluence(conservativeness=1)

# QE semantics
# agg_f = grad.semantics.modular.SumAggregation()
# inf_f = grad.semantics.modular.QuadraticMaximumInfluence(conservativeness=1)

strength_values = grad.algorithms.computeStrengthValues(bag, agg_f, inf_f)

In [5]:
# get all the attacks and supports in QBAF
att_relation = bag.get_attacks()
sup_relation = bag.get_supports()
all_relation = att_relation + sup_relation

In [6]:
# rename relation concisely
all_relation_new = []
for i in all_relation:
    if str(i)[0:3]=="Att":
        all_relation_new.append(("att",str(i.get_attacker().get_name()),str(i.get_attacked().get_name())))
    elif str(i)[0:3]=="Sup":
        all_relation_new.append(("sup",str(i.get_supporter().get_name()),str(i.get_supported().get_name())))

In [7]:
topic_a = "1"
topic_arg = bag.arguments[topic_a]

In [8]:
# compute strength for all arguments
for arg in bag.arguments.values():
    print((arg.name,arg.initial_weight,arg.strength))

('1', 0.5, 0.2543945275247097)
('2', 0.5, 0.5009765550494194)
('3', 0.5, 0.9921875)
('4', 0.5, 0.9999999850988388)
('5', 0.5, 0.998046875)
('6', 0.5, 0.5)
('7', 0.5, 0.5)
('8', 0.5, 0.5)
('9', 0.5, 0.5)
('10', 0.5, 0.5)
('11', 0.5, 0.5)
('12', 0.5, 0.99951171875)
('13', 0.5, 0.99609375)
('14', 0.5, 0.984375)
('15', 0.5, 0.75)
('16', 0.5, 0.984375)
('17', 0.5, 0.96875)
('18', 0.5, 0.96875)
('19', 0.5, 0.875)
('20', 0.5, 0.9375)
('21', 0.5, 0.96875)
('22', 0.5, 0.5)
('23', 0.5, 0.75)
('24', 0.5, 0.875)
('25', 0.5, 0.9375)
('26', 0.5, 0.9375)
('27', 0.5, 0.5)
('28', 0.5, 0.5)
('29', 0.5, 0.5)
('30', 0.5, 0.5)
('31', 0.5, 0.5)
('32', 0.5, 0.5)
('33', 0.5, 0.875)
('34', 0.5, 0.5)
('35', 0.5, 0.5)
('36', 0.5, 0.5)
('37', 0.5, 0.5)
('38', 0.5, 0.5)
('39', 0.5, 0.5)
('40', 0.5, 0.5)
('41', 0.5, 0.5)
('42', 0.5, 0.5)
('43', 0.5, 0.5)
('44', 0.5, 0.5)
('45', 0.5, 0.5)
('46', 0.5, 0.5)
('47', 0.5, 0.5)
('48', 0.5, 0.5)


In [9]:
# approximate attributions
n = len(all_relation_new) # the number of all relations in the AF
attribution = {} # store the attribution of all the relations
sample_count_N = 1000 # iterate 1000 times

for i in range(len(all_relation_new)):
    print(f"edge i:{i}")

    # split current and the other relation
    cur_relation = all_relation_new[i]
    rest_relation = all_relation_new.copy()
    rest_relation.remove(cur_relation)

    difference = []
    coefficient = []

    sample_count = sample_count_N

    for index in range(1, sample_count+1): # iterations
        new_list = generate_new_list(rest_relation) # choose random elements in rest_relation
        if new_list == []:
            new_tuple = ()
        else:
            new_tuple = tuple(new_list)
        j = new_tuple
        for k in j:
            # remove att
            if k[0] == "att":
                temp_attacker = bag.remove_attack(k[1],k[2]) # k[1] attacks k[2]
            # remove sup
            elif k[0] == "sup":
                temp_supporter = bag.remove_support(k[1],k[2]) # k[1] supports k[2]



        # compute sigma(S U {i})
        strength_values = grad.algorithms.computeStrengthValues(bag, agg_f, inf_f)
        topic_arg = bag.arguments[topic_a]
        strength_before_drop_cur = topic_arg.strength

        # remove cur_relation
        if cur_relation[0]=="att":
            temp_attacker = bag.remove_attack(cur_relation[1],cur_relation[2])
        elif cur_relation[0]=="sup":
            temp_supporter = bag.remove_support(cur_relation[1],cur_relation[2])

        # compute sigma(S)
        strength_values = grad.algorithms.computeStrengthValues(bag, agg_f, inf_f)
        topic_arg = bag.arguments[topic_a]
        strength_after_drop_cur = topic_arg.strength

        # compute sigma(S U {i})-sigma(S)
        difference.append(strength_before_drop_cur-strength_after_drop_cur)

        # recover BAG for the next iteration
        bag = grad.BAG("../../bags/fig6.bag")
        strength_values = grad.algorithms.computeStrengthValues(bag, agg_f, inf_f)

    # compute the attribution
    attribution[cur_relation] = sum(difference) / sample_count_N

edge i:0
edge i:1
edge i:2
edge i:3
edge i:4
edge i:5
edge i:6
edge i:7
edge i:8
edge i:9
edge i:10
edge i:11
edge i:12
edge i:13
edge i:14
edge i:15
edge i:16
edge i:17
edge i:18
edge i:19
edge i:20
edge i:21
edge i:22
edge i:23
edge i:24
edge i:25
edge i:26
edge i:27
edge i:28
edge i:29
edge i:30
edge i:31
edge i:32
edge i:33
edge i:34
edge i:35
edge i:36
edge i:37
edge i:38
edge i:39
edge i:40
edge i:41
edge i:42
edge i:43
edge i:44
edge i:45
edge i:46


In [10]:
attribution

{('att', '3', '1'): -0.4553359375,
 ('att', '5', '2'): -0.09130078125,
 ('att', '43', '32'): -4.064750671386719e-05,
 ('att', '44', '32'): -5.818748474121094e-05,
 ('sup', '2', '1'): 0.25962406539916993,
 ('sup', '4', '2'): 0.11530842876434326,
 ('sup', '6', '3'): -0.01514453125,
 ('sup', '7', '3'): -0.0159453125,
 ('sup', '8', '3'): -0.0164140625,
 ('sup', '9', '3'): -0.013234375,
 ('sup', '10', '3'): -0.01462109375,
 ('sup', '11', '3'): -0.0151484375,
 ('sup', '12', '4'): 0.009594302177429199,
 ('sup', '13', '4'): 0.00935735321044922,
 ('sup', '14', '4'): 0.008407132148742676,
 ('sup', '15', '5'): -0.01232763671875,
 ('sup', '16', '5'): -0.01747119140625,
 ('sup', '17', '12'): 0.0017309073209762574,
 ('sup', '18', '12'): 0.0012743062973022462,
 ('sup', '19', '13'): 0.0013789553642272949,
 ('sup', '20', '13'): 0.0015748465061187745,
 ('sup', '21', '14'): 0.0017821805477142333,
 ('sup', '22', '15'): -0.00230615234375,
 ('sup', '23', '16'): -0.0015986328125,
 ('sup', '24', '16'): -0.002